# Countries


In [3]:
countries = pd.read_csv('PreLoad/countries.txt')
countries.columns = ['country', 'code', 'un_code']
countries.set_index('un_code', inplace=True)
countries.to_sql('master_countries', con=con, if_exists='append', index=True, index_label='un_code')

# Provinces

## Canada

In [6]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "terminus.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
import pandas as pd
import numpy as np
from master.models import *

def add_provinces(country_name, provice_list, code_list):
    cnt = Countries.objects.get(country=country_name)
    for i, k in zip(provice_list, code_list):
        x = Provinces(province=i, code=k, country=cnt)
        x.save()

def add_city(provinces_codes, city_list, single_state=False):
    if single_state:
        prov = Provinces.objects.get(code=provinces_codes)
        for i in city_list:
            x = Cities(city=i, province=prov)
            x.save()
    else:
        for i, k in zip(provinces_codes, city_list):
            prov = Provinces.objects.get(code=i)
            x = Cities(city=k, province=prov)
            x.save()

def add_address(unit, address, city, zip_code):
    x = Address(unit=unit, address=address, city=Cities.objects.get(city=city), zip_code=zip_code)
    x.save()

In [5]:
# Canada
provinces = ['Alberta', 'British Columbia']
codes = ['AB', 'BC']
add_provinces("Canada", provinces, codes)

# Mexico
state = ['Mexico City']
code = ['CMX']
add_provinces('Mexico', state, code)

# India
India = pd.read_csv('PreLoad\states.csv', sep='\t')
India = India[India['Subdivision category'] == 'State']
India['Subdivision name'] = India['Subdivision name'].str.replace(r"\[(?:.*)]", "")
add_provinces('India', ['Delhi', 'Uttarakhand'], ['DL', 'UT'])

<ipython-input-5-002ae7527fb6>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  India['Subdivision name'] = India['Subdivision name'].str.replace(r"\[(?:.*)]", "")


# Cities

In [6]:
add_city('BC', ['Vancouver', 'Surrey', 'Delta', 'Richmond'], single_state=True)
add_city(['CMX'], ['Mexico City'])
add_city(['UT'], ['Mussoorie'])
add_city(['DL'], ['Delhi'])

# Addresses

In [7]:
addresses = pd.read_csv("PreLoad/Addresses.csv").loc[:11].drop(9)
addresses.columns = ['Start', 'End', 'Unit', 'Address', 'District', 'City', 'Province', 'Zip', 'Country']
addresses = addresses.fillna(np.nan).replace([np.nan], [None])

for i in addresses.iterrows():
    i = i[1]
    x = Address(unit=i.Unit, address=i.Address, district=i.District, city=Cities.objects.get(city=i.City), zip_code=i.Zip)
    x.save()


# Housing

In [2]:
def add_housing(zip_code, start, end):
    x = Housing(address=Address.objects.get(zip_code=zip_code), from_date=start, to_date=end)
    x.save()

housing = pd.read_csv("PreLoad/Addresses.csv").loc[:11].drop(9)
housing['Start'] = pd.to_datetime(housing['Start'], format='%Y-%m-%d')
housing['End'] = pd.to_datetime(housing['End'], format='%Y-%m-%d')

for h in housing.loc[:9].iterrows():
    h = h[1]
    add_housing(h.Zip, h.Start, h.End)

# Employers

In [5]:
def add_employer(employer, address, phone):
    x = Employer(employer=employer, address=Address.objects.get(address=address), phone=phone)
    x.save()

add_address(unit=None, address='7800 Alpha Way', city='Delta', zip_code='V4k 0A7')
add_address(unit='Unit 4', address='4335 Skeena Street', city='Delta', zip_code='V4k 0A6')
add_address(unit=None, address='4440 Cowley Crescent', city='Richmond', zip_code='V7B 1B8')
add_employer(employer='Alpha Aviation, Inc', address='7800 Alpha Way', phone=6049465361)
add_employer('Pacific Flying Club', '4335 Skeena Street', 6049460011)
add_employer("Pacific Coastal Airlines", '4440 Cowley Crescent', 6042142361)

# Work

In [6]:
def add_work(position, employer, start, end):
    x = Work(position=position, employer=Employer.objects.get(employer=employer), start_date=start, end_date=end)
    x.save()

w = pd.read_csv('PreLoad\work.csv').dropna()
w['Start'] = pd.to_datetime(w['Start'], format='%Y-%m-%d')
w.End = pd.to_datetime(w.End, format='%Y-%m-%d')
position = ['First Officer', 'Flight Instructor', 'CSA']
w['Position'] = position
for work in w.iterrows():
    work = work[1]
    add_work(work.Position, work.Employer, work.Start, work.End)

# Travel

In [7]:
def add_travel(from_country, to_country, travel_date, comments):
    x = Travel(from_country=Countries.objects.get(code=from_country), to_country=Countries.objects.get(code=to_country), travel_date=travel_date, comments=comments)
    x.save()

t = pd.read_csv('PreLoad/travel.csv')
t = t.fillna(np.nan).replace([np.nan], [None])
t.Date = pd.to_datetime(t.Date, format='%Y-%m-%d')
t = t.loc[:56]

for travel in t.iterrows():
    travel = travel[1]
    add_travel(travel.Dep, travel.Arr, travel.Date, travel.Comments)

# Authors

In [5]:
import pandas as pd
import numpy as np
import sqlite3

con = sqlite3.connect('master.sqlite3')
cb = pd.read_csv('PreLoad/Books_complete.csv')
authors = pd.Series(cb['Author(s)'].unique()).str.split(",", expand=True)
authors.columns = ['surname', 'first_name']
authors['surname'] = authors['surname'].str.strip()
authors['first_name'] = authors['first_name'].str.strip()
authors.drop_duplicates(inplace=True)
authors.sort_values(by='surname', inplace=True)
authors = authors.fillna(np.nan).replace([np.nan], [None])
authors.to_sql('master_authors', con=con, if_exists='append', index=True, index_label='id')

# Books

In [3]:
import pandas as pd
import numpy as np

In [10]:
def add_books(title, series, book_format, author_first, author_last):
    try:
        book = Books.objects.get(title=title, series=series, book_format=book_format)
    except Exception as e:
        print(e)
        print(f'Adding new book, {title}, {book_format}, {author_first, author_last}')
        book = Books(title=title, series=series, book_format=book_format)
        book.save()
    book.authors.add(Authors.objects.get(first_name=author_first, surname=author_last))

b = pd.read_csv('PreLoad/Books_complete.csv')
b[['last_name', 'first_name']] = pd.Series(b['Author(s)']).str.split(",", expand=True)
b = b.fillna(np.nan).replace([np.nan], [None])
b.last_name = b.last_name.str.strip()
b.first_name = b.first_name.str.strip()
b.Format = b.Format.replace([None], 'print')
b = b.rename({'Read?':'Read'}, axis=1)

In [11]:
b

,Name,Author(s),Read,Format,Series,Editor(s),Translator(s),Arr,last_name,first_name
0,The New Jim Crow: Mass Incarceration in the Ag...,"Alexander, Michelle",N,print,None,None,None,None,Alexander,Michelle
1,London Fields,"Amis, Martin",S,print,None,None,None,None,Amis,Martin
2,Modern Philosophy: An Anthology of Primary Sou...,"Watkins, Eric",S,print,None,"Ariew, Roger; Watkins, Eric",None,None,Watkins,Eric
3,Modern Philosophy: An Anthology of Primary Sou...,"Ariew, Roger",S,print,None,"Ariew, Roger; Watkins, Eric",None,None,Ariew,Roger
4,The Metaphysics,"Aristotle,",S,print,None,None,None,None,Aristotle,
...,...,...,...,...,...,...,...,...,...,...
217,Children of Time,"Tchaikovsky, Adrian",Y,Audiobook,None,None,None,None,Tchaikovsky,Adrian
218,The Bomber Mafia,"Gladwell, Malcolm",N,Audiobook,None,None,None,None,Gladwell,Malcolm
219,The Hitchhiker's Guide to the Galaxy,"Adams, Douglas",Y,Audiobook,None,None,None,None,Adams,Douglas
220,If Toy Tell,"Olsen, Gregg",N,Audiobook,None,None,None,None,Olsen,Gregg


In [9]:
for entry in b.iterrows():
    entry = entry[1]
    add_books(title=entry.Name, series=entry.Series, book_format=entry.Format, author_first=entry.first_name, author_last=entry.last_name)

Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
Books matching query does not exist.
B

# Documents

In [7]:
import pandas as pd

d = pd.read_csv('PreLoad/Documents.csv').dropna()
d.Expiration = pd.to_datetime(d.Expiration, format='%d-%m-%y')

In [5]:
d

,Expiration,Item
0,01-12-23,Amex
1,01-08-24,Aviation Booklet
2,01-06-21,Aviation Medical
3,01-11-22,Banamex Debit
4,06-09-20,Dangerous Goods Card
5,21-02-22,Driver's License
6,19-07-21,Elementary Work Authority
7,01-01-21,IFE
8,21-02-22,MSP Card
9,01-01-21,OCI


In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "terminus.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
import pandas as pd
import numpy as np
from master.models import Documents

In [2]:
documents = Documents.objects.order_by('expiry')

In [3]:
for doc in documents:
    print(doc.document)

Dangerous Goods Card
Passport
IFE
OCI
Aviation Medical
Elementary Work Authority
Driver's License
MSP Card
Banamex Debit
Scotia Debit
TD Debit
Tangerine Mastercard
Amex
Scotia LOC
PR
Aviation Booklet
Scotia Visa
Tangerine Debit
US Visa


In [10]:
from datetime import datetime, timedelta
for doc in Documents.objects.filter(expiry__lt=(datetime.now() + timedelta(days=1))):
    print(doc.document)

Aviation Medical
Dangerous Goods Card
Elementary Work Authority
IFE
OCI
Passport
